# Analyze metadata from Zenodo spreadsheets corpus
version of the notebook from March 2025

In [1]:
import re
from datetime import datetime
import json
import pandas as pd
import math
import numpy as np
import edtf
from decimal import Decimal
import matplotlib.pyplot as plt

In [ ]:
#https://www.askpython.com/python/dictionary/create-nested-dictionary-via-for-loop
#extract file information from API response, adding some record-level metadata to each file
#create a list of dictionaries
#calculate the total size of all files
file_sizes = []
with open('/Volumes/ap180/zenodo_API_metadata_results_2025-01-08_01-46_PM.json', 'r') as f:
#with open('zenodo_snippet.json', 'r') as f:
    data = f.read()

filesize_total = 0
zenodo_dict = json.loads(data)
for rec in zenodo_dict:
    #print(rec['files'])
    files = rec['files']
    record_id = rec['id']
    date = rec['metadata']['publication_date']
    for file in files:
        match = re.match(r'.*(\.xls[xb]?|\.csv|\.tsv|\.ods)$',file['key'])
        #
        #print(match)
        file_info = {}
        if match:
            file_info['recordid'] = record_id
            file_info['date'] = date
            file_info['filename'] = file['key']
            file_info['filetype'] = match.group(1)
            filesize = int(file['size'])
            file_info['filesize_b'] = filesize
            filesize_total = filesize_total + filesize
            file_sizes.append(file_info)
        else:
            pass


In [ ]:
#https://www.knowprogram.com/python/bytes-to-kb-mb-gb-and-tb-in-python/
#https://blog.finxter.com/5-best-ways-to-format-bytes-to-gigabytes-in-python/
#show the total size of all files, displaying in bytes, TB, and GB
print(filesize_total)
terabytes_size = Decimal(filesize_total) / Decimal(1024 * 1024 * 1024 * 1024)
formatted_tb_size = "{:.2f} TB".format(terabytes_size)
gigabytes_size = Decimal(filesize_total) / Decimal(1024 ** 3)
formatted_gb_size = f"{gigabytes_size:.2f} GB"
print(formatted_tb_size)
print(formatted_gb_size)

In [ ]:
#first round results
#8223405220559
#7.48 TB
#7658.64 GB


In [ ]:
#create a data frame from the list of dictionaries
filesize_df = pd.DataFrame(file_sizes)
    
filesize_df.head(5)


In [ ]:
filesize_df['filetype'].str.contains('.xlsb').any()

In [ ]:
filesize_df.groupby(['filetype'])['filetype'].value_counts()

In [ ]:
#add columns to dataframe that convert size in bytes to MB and GB respectively
filesize_df['filesize_mb'] = filesize_df['filesize_b'].apply(lambda x: x / pow(1024, 2))
filesize_df['filesize_gb'] = filesize_df['filesize_b'].apply(lambda x: x / pow(1024, 3))
filesize_df

In [ ]:
# total count of rows
filesize_df.shape[0]

In [ ]:
#get some stats about each file type
#https://www.geeksforgeeks.org/pandas-groupby-one-column-and-get-mean-min-and-max-values/
filesize_df.groupby(['filetype']).agg({'filesize_mb': ['mean', 'min', 'max', 'median', 'count','sum']})

In [ ]:
filesize_df.groupby(['filetype']).agg({'filesize_gb': ['sum']})

In [ ]:
#https://www.statology.org/pandas-groupby-plot/
#first attempts at plotting
filesize_df.groupby('filetype')['filesize_mb'].plot(legend=True)

In [ ]:
filesize_df.groupby('filetype')['filesize_mb'].value_counts().plot(legend=True)
filesize_df.groupby('filetype')['filesize_mb'].value_counts()

In [ ]:
#create groups
filesize_groups = filesize_df.groupby('filetype')

In [ ]:
csv_group = filesize_groups.get_group('.csv')
csv_group

In [ ]:
tsv_group = filesize_groups.get_group('.tsv')
tsv_group

In [ ]:
ods_group = filesize_groups.get_group('.ods')
ods_group

In [ ]:
xlsx_group = filesize_groups.get_group('.xlsx')
xlsx_group

In [ ]:
xls_group = filesize_groups.get_group('.xls')
xls_group

In [ ]:
xlsb_group = filesize_groups.get_group('.xlsb')
xlsb_group

In [ ]:
#https://github.com/pandas-dev/pandas/issues/28298
#filesize_groups.get_group(('.xlsx', '.xlsb'))
grpidx=('.xlsx', '.xlsb', '.xls', '.ods')
dfidx=np.sort(np.concatenate([filesize_groups.indices[x] for x in grpidx]))
excel_ods_df = filesize_df.loc[dfidx]

In [ ]:
excel_ods_df.groupby(['filetype']).agg({'filesize_gb': ['sum','count']})

In [ ]:
#sample xlsx group
xlsx_sample = xlsx_group.sample(n=100, random_state=1)

In [ ]:
xlsx_sample.agg({'filesize_mb': ['mean', 'min', 'max', 'median', 'count','sum']})

In [ ]:
xlsx_sample

In [ ]:
xlsx_sample['recordid']

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

xlsx_sample['recordid'].to_csv(f'xlsx_sample_{filetime}.csv', index=False)

In [ ]:
xlsx_sample_series = xlsx_sample['recordid']

In [ ]:
xlsx_sample_series.info()

In [ ]:
#https://stackoverflow.com/questions/12096252/use-a-list-of-values-to-select-rows-from-a-pandas-dataframe
files_in_xlsx_sample = xlsx_group.query('recordid in @xlsx_sample_series')

In [ ]:
files_in_xlsx_sample

In [ ]:
files_in_xlsx_sample['filetype']=='.xlsx'

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

xlsx_group['recordid'].to_csv(f'xlsx_group_{filetime}.csv', index=False)

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

excel_ods_df['recordid'].to_csv(f'excel_ods_{filetime}.csv', index=False)

In [ ]:
xlsb_group['filesize_mb'].plot(kind='box')

In [ ]:
#https://python.plainenglish.io/identifying-and-handling-outliers-in-pandas-a-step-by-step-guide-fcecd5c6cd3b
#excluding outliers https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-a-pandas-dataframe
#creating some new dataframes and columns to detect and exclude outliers based on z-scores
z_scores_filesize_mb = np.abs((filesize_df['filesize_mb'] - filesize_df['filesize_mb'].mean()) / filesize_df['filesize_mb'].std())
z_scores_filesize_mb

In [ ]:
filesize_df['z_score_filesize_mb'] = z_scores_filesize_mb

In [ ]:
filesize_df

In [ ]:
# Define threshold (here: remove values with z-score > 2)
threshold = 2

# create DF of outliers based on z-score and threshold
filesize_df_zscore_outliers = filesize_df[filesize_df['z_score_filesize_mb'] >= threshold]
filesize_df_zscore_outliers

In [ ]:
#info about the outliers (identified by z-score) specifically
filesize_df_zscore_outliers.groupby('filetype')['filesize_mb'].describe()

In [ ]:
# create DF sans outliers based on z-score and threshold
filesize_df_zscore_sans_outliers = filesize_df[filesize_df['z_score_filesize_mb'] <= threshold]
filesize_df_zscore_sans_outliers

In [ ]:
#creating some new dataframes and columns to detect and exclude outliers based on IQR

# Calculate 25% percentile and 75% percentile
Q1 = filesize_df['filesize_mb'].quantile(0.25)
Q3 = filesize_df['filesize_mb'].quantile(0.75)

# Calculate Interquartile Range (IQR)
IQR = Q3 - Q1
IQR

In [ ]:
# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
lower_bound

In [ ]:
upper_bound = Q3 + 1.5 * IQR
upper_bound

In [ ]:
# Remove outliers based on lower & upper bounds
filesize_df_iqr_sans_outliers = filesize_df[(filesize_df['filesize_mb'] >= lower_bound) & 
                        (filesize_df['filesize_mb'] <= upper_bound)]

In [ ]:
filesize_df_iqr_sans_outliers

In [ ]:
# Create a df of just the IQR outliers
filesize_df_iqr_outliers = filesize_df[(filesize_df['filesize_mb'] > upper_bound)]
filesize_df_iqr_outliers

In [ ]:
filesize_df_iqr_sans_outliers.groupby('filetype')['filesize_mb'].plot(kind='hist', bins=50, legend=True)

In [ ]:
filesize_df_iqr_sans_outliers.groupby('filetype')['filesize_mb'].describe()

In [ ]:
filesize_df_zscore_sans_outliers.groupby('filetype')['filesize_mb'].plot(kind='hist', bins=50, legend=True)

In [ ]:
filesize_df_zscore_sans_outliers.groupby('filetype')['filesize_mb'].describe()

In [ ]:
filesize_groups_zscore_sans_outliers = filesize_df_zscore_sans_outliers.groupby('filetype')

In [ ]:
filesize_groups_zscore_sans_outliers.groups

In [ ]:
ods_group_zscore_sans_outliers = filesize_groups_zscore_sans_outliers.get_group('.ods')
ods_group_zscore_sans_outliers

In [ ]:
xlsx_group_zscore_sans_outliers = filesize_groups_zscore_sans_outliers.get_group('.xlsx')
xlsx_group_zscore_sans_outliers

In [ ]:
xls_group_zscore_sans_outliers = filesize_groups_zscore_sans_outliers.get_group('.xls')
xls_group_zscore_sans_outliers

In [ ]:
xlsb_group_zscore_sans_outliers = filesize_groups_zscore_sans_outliers.get_group('.xlsb')
xlsb_group_zscore_sans_outliers

In [ ]:
tsv_group_zscore_sans_outliers = filesize_groups_zscore_sans_outliers.get_group('.tsv')
tsv_group_zscore_sans_outliers

In [ ]:
csv_group_zscore_sans_outliers = filesize_groups_zscore_sans_outliers.get_group('.csv')
csv_group_zscore_sans_outliers

In [ ]:
# csv_group_zscore_sans_outliers['filesize_mb'].plot(kind='hist', bins=50, legend=True)
# tsv_group_zscore_sans_outliers['filesize_mb'].plot(kind='hist', bins=50, legend=True)
# ods_group_zscore_sans_outliers['filesize_mb'].plot(kind='hist', bins=50, legend=True)
# xls_group_zscore_sans_outliers['filesize_mb'].plot(kind='hist', bins=50, legend=True)
# xlsx_group_zscore_sans_outliers['filesize_mb'].plot(kind='hist', bins=50, legend=True)
# xlsb_group_zscore_sans_outliers['filesize_mb'].plot(kind='hist', bins=50, legend=True)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(6,1,figsize=(12,25))

ax[0].hist(csv_group_zscore_sans_outliers['filesize_mb'], bins=50)
ax[0].set_title('CSV size in MB')
ax[1].hist(tsv_group_zscore_sans_outliers['filesize_mb'], bins=50)
ax[1].set_title('TSV size in MB')
ax[2].hist(ods_group_zscore_sans_outliers['filesize_mb'], bins=50)
ax[2].set_title('ODS size in MB')
ax[3].hist(xls_group_zscore_sans_outliers['filesize_mb'], bins=50)
ax[3].set_title('XLS size in MB')
ax[4].hist(xlsx_group_zscore_sans_outliers['filesize_mb'], bins=50)
ax[4].set_title('XLSX size in MB')
ax[5].hist(xlsb_group_zscore_sans_outliers['filesize_mb'], bins=50)
ax[5].set_title('XLSB size in MB')

In [ ]:
filesize_groups_iqr_sans_outliers = filesize_df_iqr_sans_outliers.groupby('filetype')

In [ ]:
filesize_groups_zscore_sans_outliers.groups

In [ ]:
ods_group_iqr_sans_outliers = filesize_groups_iqr_sans_outliers.get_group('.ods')
ods_group_iqr_sans_outliers

In [ ]:
csv_group_iqr_sans_outliers = filesize_groups_iqr_sans_outliers.get_group('.csv')
csv_group_iqr_sans_outliers

In [ ]:
tsv_group_iqr_sans_outliers = filesize_groups_iqr_sans_outliers.get_group('.tsv')
tsv_group_iqr_sans_outliers

In [ ]:
xlsx_group_iqr_sans_outliers = filesize_groups_iqr_sans_outliers.get_group('.xlsx')
xlsx_group_iqr_sans_outliers

In [ ]:
xls_group_iqr_sans_outliers = filesize_groups_iqr_sans_outliers.get_group('.xls')
xls_group_iqr_sans_outliers

In [ ]:
xlsb_group_iqr_sans_outliers = filesize_groups_iqr_sans_outliers.get_group('.xlsb')
xlsb_group_iqr_sans_outliers

In [ ]:
fig, ax = plt.subplots(6,1,figsize=(12,25))

ax[0].hist(csv_group_iqr_sans_outliers['filesize_mb'], bins=50)
ax[0].set_title('CSV size in MB')
ax[1].hist(tsv_group_iqr_sans_outliers['filesize_mb'], bins=50)
ax[1].set_title('TSV size in MB')
ax[2].hist(ods_group_iqr_sans_outliers['filesize_mb'], bins=50)
ax[2].set_title('ODS size in MB')
ax[3].hist(xls_group_iqr_sans_outliers['filesize_mb'], bins=50)
ax[3].set_title('XLS size in MB')
ax[4].hist(xlsx_group_iqr_sans_outliers['filesize_mb'], bins=50)
ax[4].set_title('XLSX size in MB')
ax[5].hist(xlsb_group_iqr_sans_outliers['filesize_mb'], bins=50)
ax[5].set_title('XLSB size in MB')

In [ ]:
csv_group_sans_outliers

In [ ]:
filesize_df_zscore_outliers[filesize_df_zscore_outliers['filetype']=='.tsv']

In [ ]:
plt.hist(filesize_df_zscore_outliers['filesize_mb'], bins=50)

In [ ]:
plt.hist(filesize_df_zscore_outliers[filesize_df_zscore_outliers['filetype']=='.tsv'].filesize_mb, bins=50)

In [ ]:
plt.hist(filesize_df_zscore_outliers[filesize_df_zscore_outliers['filetype']=='.xlsx'].filesize_mb, bins=50)

In [ ]:
filesize_df.groupby(['filetype'])['filesize_gb'].sum()

In [ ]:
filesize_df_zscore_sans_outliers['filesize_gb'].sum()

In [ ]:
filesize_df_iqr_sans_outliers['filesize_gb'].sum()

In [ ]:
filesize_df_iqr_outliers['filesize_gb'].sum()

In [ ]:
filesize_df_zscore_outliers['filesize_gb'].sum()

In [ ]:
#want to now try to take a random sample from each group, calculate the size of each sample and the total size to see if that gets me under 5 TB
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.sample.html#pandas.core.groupby.DataFrameGroupBy.sample

In [ ]:
filesize_df_sample = filesize_df.groupby(['filetype']).sample(frac=0.75, random_state=1)
filesize_df_sample

In [ ]:
filesize_df_sample.groupby(['filetype']).agg({'filesize_gb': ['sum']})

In [ ]:
filesize_df_sample['filesize_gb'].sum()

In [ ]:
filesize_df_sample.describe()

In [ ]:
filesize_df_sample.groupby(['filetype']).agg({'filesize_mb': ['mean', 'min', 'max', 'median', 'count']})

In [ ]:
filesize_df_sample2 = filesize_df.groupby(['filetype']).sample(frac=0.75, random_state=0)
filesize_df_sample2

In [ ]:
filesize_df_sample2.groupby(['filetype']).agg({'filesize_mb': ['mean', 'min', 'max', 'median', 'count']})

In [ ]:
filesize_df_sample2['filesize_gb'].sum()

In [ ]:
filesize_df_sample3 = filesize_df.groupby(['filetype']).sample(frac=0.75, random_state=0)
filesize_df_sample3

In [ ]:
filesize_df_sample3.groupby(['filetype']).agg({'filesize_mb': ['mean', 'min', 'max', 'median', 'count']})

In [ ]:
filesize_df_sample3['filesize_gb'].sum()

In [ ]:
filesize_df_sample4 = filesize_df.groupby(['filetype']).sample(frac=0.60, random_state=1)
filesize_df_sample4

In [ ]:
filesize_df_sample4.groupby(['filetype']).agg({'filesize_mb': ['mean', 'min', 'max', 'median', 'count']})

In [ ]:
filesize_df_sample4['filesize_gb'].sum()

In [ ]:
# get just the filename, no extension
#https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html
filesize_df['filename_noext'] = filesize_df['filename'].str.extract(r'(.*)(?:\.xls[xb]?|\.csv|\.tsv|\.ods)')

In [ ]:
filesize_df_sorted = filesize_df.sort_values(by=['recordid','filename_noext','filetype'])

In [ ]:
#https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
filesize_df_sorted.groupby(['recordid','filename_noext'], sort=False).agg({'filename_noext':'count', 'filetype': ', '.join})

In [ ]:
filename_counts = filesize_df_sorted.groupby(['recordid','filename_noext'], sort=False).agg({'filename_noext':'count', 'filetype': ', '.join})

In [ ]:
filename_counts.columns

In [ ]:
filename_counts.index.name = None
filename_counts.columns=['counts', 'filetype']

In [ ]:
filename_counts

In [ ]:
filename_counts = filename_counts.rename_axis(['recordid','filename_noext']).reset_index()

In [ ]:
filename_counts

In [ ]:
filename_duplicates = filename_counts.loc[filename_counts['counts']>1]

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

filename_duplicates.to_csv(f'filename_duplicates_{filetime}.csv', index=False)

In [ ]:
#https://japblog.chickenkiller.com/blog/how-to-expand-a-nested-dictionary-in-pandas-column

In [ ]:
excel_ods_df

In [ ]:
excel_ods_df.groupby(['recordid']).agg({'recordid': ['count']})

In [ ]:
excel_ods_df_counts = excel_ods_df.groupby(['recordid']).agg({'recordid': ['count']})

In [ ]:
#https://stackoverflow.com/questions/36073504/create-entirely-new-dataframe-efficiently-from-groupby-agg-or-apply-in-pan
excel_ods_df_counts.index.name = None
excel_ods_df_counts.columns=['counts']

In [ ]:
excel_ods_df_counts.info()

In [ ]:
excel_ods_df_counts.columns

In [ ]:
excel_ods_df_counts.sort_values(by='counts')

In [ ]:
excel_ods_df_counts_sample = excel_ods_df_counts.sample(n=100, random_state=2)

In [ ]:
excel_ods_df_counts_sample.sum()

In [ ]:
excel_ods_df_counts_sample.sort_values(by='counts')

In [ ]:
xlsx_group_counts = xlsx_group.groupby(['recordid']).agg({'recordid': ['count']})

In [ ]:
xlsx_group_counts

In [ ]:
xlsx_group_counts.index.name = None
xlsx_group_counts.columns=['counts']

In [ ]:
xlsx_group_counts.sort_values(by='counts')

In [ ]:
xlsx_group_counts_sample = xlsx_group_counts.sample(n=100, random_state=2)

In [ ]:
xlsx_group_counts_sample.sum()

In [ ]:
#https://stackoverflow.com/questions/20461165/how-to-convert-index-of-a-pandas-dataframe-into-a-column
xlsx_group_counts_sample=xlsx_group_counts_sample.rename_axis('recordid').reset_index()

In [ ]:
xlsx_group_counts_sample.columns

In [ ]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

xlsx_group_counts_sample['recordid'].to_csv(f'xlsx_group_counts_sample_{filetime}.csv', index=False)

In [ ]:
filesize_df.loc[filesize_df['recordid'] == 10178665]

# Section for json_normalize code

In [2]:
with open('/Volumes/ap180/zenodo_API_metadata_results_2025-01-08_01-46_PM.json', 'r') as f:
#with open('zenodo_snippet.json', 'r') as f:
    data = f.read()
zenodo_dict = json.loads(data)

In [108]:
#load 450 additional data/records with additional filetypes ots OR xlsm OR xlr OR xltm OR xl OR cell OR xar OR pmd OR xltx OR imp OR ast OR dis OR ogw OR fcs
with open('/Volumes/ap180/zenodo_API_metadata_results_2025-03-21_01-34_PM.json', 'r') as f:
#with open('zenodo_snippet.json', 'r') as f:
    data_additional = f.read()
zenodo_dict_additional = json.loads(data_additional)

In [4]:
#https://stackoverflow.com/questions/52085169/valueerror-conflicting-metadata-name-name-need-distinguishing-prefix-in-pandas
zenodo_files_df = pd.json_normalize(zenodo_dict, record_path=[['files']],
                                       meta=['id'], record_prefix='file'
                                      )

In [109]:
#https://stackoverflow.com/questions/52085169/valueerror-conflicting-metadata-name-name-need-distinguishing-prefix-in-pandas
zenodo_files_additional_df = pd.json_normalize(zenodo_dict_additional, record_path=[['files']],
                                       meta=['id'], record_prefix='file'
                                      )

In [6]:
zenodo_files_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804830 entries, 0 to 804829
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   fileid          804830 non-null  object
 1   filekey         804830 non-null  object
 2   filesize        804830 non-null  int64 
 3   filechecksum    804830 non-null  object
 4   filelinks.self  804830 non-null  object
 5   id              804830 non-null  object
dtypes: int64(1), object(5)
memory usage: 36.8+ MB


In [110]:
zenodo_files_additional_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3059 entries, 0 to 3058
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   fileid          3059 non-null   object
 1   filekey         3059 non-null   object
 2   filesize        3059 non-null   int64 
 3   filechecksum    3059 non-null   object
 4   filelinks.self  3059 non-null   object
 5   id              3059 non-null   object
dtypes: int64(1), object(5)
memory usage: 143.5+ KB


In [111]:
#https://www.geeksforgeeks.org/concatenate-pandas-dataframes-without-duplicates/
zenodo_files_all_df = pd.concat([zenodo_files_df, zenodo_files_additional_df])

In [112]:
zenodo_files_all_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 807889 entries, 0 to 3058
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   fileid          807889 non-null  object
 1   filekey         807889 non-null  object
 2   filesize        807889 non-null  int64 
 3   filechecksum    807889 non-null  object
 4   filelinks.self  807889 non-null  object
 5   id              807889 non-null  object
dtypes: int64(1), object(5)
memory usage: 43.1+ MB


In [113]:
#https://stackoverflow.com/questions/28885073/resetting-index-after-calling-pandas-drop-duplicates
zenodo_files_all_df = zenodo_files_all_df.drop_duplicates().reset_index(drop=True)

In [114]:
zenodo_files_all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 794648 entries, 0 to 794647
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   fileid          794648 non-null  object
 1   filekey         794648 non-null  object
 2   filesize        794648 non-null  int64 
 3   filechecksum    794648 non-null  object
 4   filelinks.self  794648 non-null  object
 5   id              794648 non-null  object
dtypes: int64(1), object(5)
memory usage: 36.4+ MB


In [115]:
# get just the filenames and extensions
#https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html
zenodo_files_all_df['filetype'] = zenodo_files_all_df['filekey'].str.extract(r'(?:.*)(\..+)$')

In [116]:
zenodo_files_all_df['filetype']

0          .xls
1          .pdf
2          .ods
3         .xlsx
4          .ods
          ...  
794643     .zip
794644    .xlsm
794645     .rar
794646     .rar
794647    .xlsm
Name: filetype, Length: 794648, dtype: object

In [117]:
zenodo_files_all_df['filename_noext'] = zenodo_files_all_df['filekey'].str.extract(r'(.*)(?:\..+)$')

In [118]:
zenodo_files_all_df['filename_noext']

0                                                database_D
1                                                     Table
2                                                database_D
3                                       migration_modelling
4                                       migration_modelling
                                ...                        
794643                                  NZ_scenario results
794644                            Input data NZ_CC_scenario
794645                                                   MC
794646                                                   BN
794647    characteristics of reservoirs and river flood ...
Name: filename_noext, Length: 794648, dtype: object

In [119]:
#add columns to dataframe that convert size in bytes to MB and GB respectively
zenodo_files_all_df['filesize_mb'] = zenodo_files_df['filesize'].apply(lambda x: x / pow(1024, 2))
zenodo_files_all_df['filesize_gb'] = zenodo_files_df['filesize'].apply(lambda x: x / pow(1024, 3))

In [120]:
zenodo_files_all_df

,fileid,filekey,filesize,filechecksum,filelinks.self,id,filetype,filename_noext,filesize_mb,filesize_gb
0,e28f3dc8-28bb-4c5e-9279-2ec8323baecd,database_D.xls,41472,md5:2bcfd1bd8dd9c49e8529bf6b1e2c0da8,https://zenodo.org/api/records/883073/files/da...,883073,.xls,database_D,0.039551,0.000039
1,97f5df9e-a949-42e5-86b6-27d64d4c71ea,Table.pdf,75939,md5:cf61aab0ad3bc61930dd2ce9617f2ae5,https://zenodo.org/api/records/883073/files/Ta...,883073,.pdf,Table,0.072421,0.000071
2,a2414a60-1598-4781-9598-c6e490c45abb,database_D.ods,24710,md5:acd49bf242066b404136f75358735f1f,https://zenodo.org/api/records/883073/files/da...,883073,.ods,database_D,0.023565,0.000023
3,c3abd72d-8ff6-47d5-b54a-2a2c8ec8a37c,migration_modelling.xlsx,1193750,md5:1ffd4ff4009cb149e92b1faed928ca32,https://zenodo.org/api/records/883073/files/mi...,883073,.xlsx,migration_modelling,1.138449,0.001112
4,dd6092e9-477f-4309-a2dd-8a3d6fc4a950,migration_modelling.ods,958046,md5:73dcaae0470108cb4d26bf279a007a8a,https://zenodo.org/api/records/883073/files/mi...,883073,.ods,migration_modelling,0.913664,0.000892
...,...,...,...,...,...,...,...,...,...,...
794643,ceacaaaf-2408-4b53-a7ef-88a2e00cf829,NZ_scenario results.zip,332991,md5:0a436a1f89c3d5763da5170a2d978aef,https://zenodo.org/api/records/7859482/files/N...,7859482,.zip,NZ_scenario results,1.516737,0.001481
794644,46d84edb-7af1-4929-ba01-5ab6c53b2c26,Input data NZ_CC_scenario.xlsm,43534008,md5:233150a881b38d7a8684acc42dbc8765,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_CC_scenario,0.075867,0.000074
794645,9de71906-462e-407b-b258-4660769fa91f,MC.rar,69176379,md5:1b2013fbd1ebfdf9fd303892af484772,https://zenodo.org/api/records/1164432/files/M...,1164432,.rar,MC,0.002212,0.000002
794646,a7db9ff2-7ef1-49ef-b504-5b2530724114,BN.rar,1718709,md5:068f6086e94cf9a5608edea8f0122fc3,https://zenodo.org/api/records/1164432/files/B...,1164432,.rar,BN,0.088968,0.000087


In [121]:
zenodo_files_all_df.groupby(['filetype']).agg({'filesize_mb': ['mean', 'min', 'max', 'median', 'count','sum']})

filesize_mb             \
                                                     mean        min   
filetype                                                               
. - PAT-GEOM - R Code                            0.001856   0.001856   
. BAU                                            3.916775   3.916775   
. Fig 3_DATA for Clustering                     42.951537  42.951537   
. Fig 4_Perseverance vs Baseline rate            0.135891   0.135891   
. Low Demand & PPA Renewed (Optimal Capacity)   48.940026  48.940026   
...                                                   ...        ...   
.zmfa                                            0.268825   0.268825   
.zmt                                             0.003556   0.002342   
.zst                                           533.406858   0.000899   
.ztt                                             0.239639   0.237573   
.zvi                                            35.551256   0.000502   

                                                                             \
                                                       max     median count   
filetype                                                                      
. - PAT-GEOM - R Code                             0.001856   0.001856     1   
. BAU                                             3.916775   3.916775     1   
. Fig 3_DATA for Clustering                      42.951537  42.951537     1   
. Fig 4_Perseverance vs Baseline rate             0.135891   0.135891     1   
. Low Demand & PPA Renewed (Optimal Capacity)    48.940026  48.940026     1   
...                                                    ...        ...   ...   
.zmfa                                             0.268825   0.268825     1   
.zmt                                              0.005858   0.002466     3   
.zst                                           4369.474188   4.634688    22   
.ztt                                              0.241935   0.239523     4   
.zvi                                            301.312713   0.014966    21   

                                                             
                                                        sum  
filetype                                                     
. - PAT-GEOM - R Code                              0.001856  
. BAU                                              3.916775  
. Fig 3_DATA for Clustering                       42.951537  
. Fig 4_Perseverance vs Baseline rate              0.135891  
. Low Demand & PPA Renewed (Optimal Capacity)     48.940026  
...                                                     ...  
.zmfa                                              0.268825  
.zmt                                               0.010667  
.zst                                           11734.950884  
.ztt                                               0.958554  
.zvi                                             746.576378  

[2562 rows x 6 columns]

In [122]:
zenodo_files_all_df.loc[zenodo_files_all_df['filetype'] == '.cell']

,fileid,filekey,filesize,filechecksum,filelinks.self,id,filetype,filename_noext,filesize_mb,filesize_gb


In [123]:
spreadsheet_filetypes = ['.csv','.tsv','.xls','.xlsb','.xlsx','.xltx','.ods','.ots','.xlsm','.xltm','.xltx','.ogw']
#probably need to remove xlr (no recs) fcs, xar (archive file), .ast (IDEAS software produced XML), pmd files - these extensions refer to other filetypes too

In [124]:
#https://stackoverflow.com/questions/19960077/how-to-filter-pandas-dataframe-using-in-and-not-in-like-in-sql
zenodo_spreadsheet_files_df = zenodo_files_all_df.query('filetype in @spreadsheet_filetypes')

In [125]:
zenodo_spreadsheet_files_df

,fileid,filekey,filesize,filechecksum,filelinks.self,id,filetype,filename_noext,filesize_mb,filesize_gb
0,e28f3dc8-28bb-4c5e-9279-2ec8323baecd,database_D.xls,41472,md5:2bcfd1bd8dd9c49e8529bf6b1e2c0da8,https://zenodo.org/api/records/883073/files/da...,883073,.xls,database_D,0.039551,0.000039
2,a2414a60-1598-4781-9598-c6e490c45abb,database_D.ods,24710,md5:acd49bf242066b404136f75358735f1f,https://zenodo.org/api/records/883073/files/da...,883073,.ods,database_D,0.023565,0.000023
3,c3abd72d-8ff6-47d5-b54a-2a2c8ec8a37c,migration_modelling.xlsx,1193750,md5:1ffd4ff4009cb149e92b1faed928ca32,https://zenodo.org/api/records/883073/files/mi...,883073,.xlsx,migration_modelling,1.138449,0.001112
4,dd6092e9-477f-4309-a2dd-8a3d6fc4a950,migration_modelling.ods,958046,md5:73dcaae0470108cb4d26bf279a007a8a,https://zenodo.org/api/records/883073/files/mi...,883073,.ods,migration_modelling,0.913664,0.000892
6,1fe5b1ee-f3bb-45ac-ac3e-a8ad82169e67,05_Supplementary_table_1.xls,896512,md5:0617b7911ab69f9426e7f58e9817b235,https://zenodo.org/api/records/12918528/files/...,12918528,.xls,05_Supplementary_table_1,0.854980,0.000835
...,...,...,...,...,...,...,...,...,...,...
794634,1fba9d0f-d7db-4d0a-a14e-637cac649a1c,Input data NZ_CO_scenario.xlsm,43534090,md5:62a7e9eb8bb84a030efd15b9fc54aaa8,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_CO_scenario,1.516737,0.001481
794638,eb5ef863-d3ba-47f6-ae32-29178d0fedbc,Input data NZ_OL_scenario.xlsm,43532603,md5:8d4640c4e355150141c1d2ff9b43b3c8,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_OL_scenario,1.152891,0.001126
794639,63fe0822-4b74-4561-a625-6cb94d18160d,Input data NZ_scenario.xlsm,43532584,md5:6e10cf247a8e489af11e8d2b1955bc68,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_scenario,1.152891,0.001126
794644,46d84edb-7af1-4929-ba01-5ab6c53b2c26,Input data NZ_CC_scenario.xlsm,43534008,md5:233150a881b38d7a8684acc42dbc8765,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_CC_scenario,0.075867,0.000074


In [126]:
zenodo_spreadsheet_files_df.groupby(['filetype']).agg({'filesize_gb': ['mean', 'min', 'max', 'median', 'count','sum']})

filesize_gb                                                         
                mean           min        max    median   count           sum
filetype                                                                     
.csv        0.062343  0.000000e+00  96.030568  0.000140  173074  10789.924066
.ods        0.020891  3.632158e-08  19.513078  0.000034    1257     26.260183
.ogw        0.000239  1.737103e-05   0.001481  0.000167      25      0.005963
.ots        0.001375  1.374792e-03   0.001375  0.001375       1      0.001375
.tsv        0.071389  2.793968e-09  33.526691  0.000174    8894    634.933069
.xls        0.034419  0.000000e+00  19.809742  0.000054    7349    252.941670
.xlsb       0.014515  6.002374e-06   0.107630  0.015286      97      1.407923
.xlsm       0.103609  1.480803e-07  18.114483  0.008886     981    101.640541
.xlsx       0.043246  0.000000e+00  54.950428  0.000050   88930   3845.889887
.xltm       0.000358  1.364201e-04   0.002478  0.000170      12      0.004299
.xltx       0.085753  1.640990e-06   1.784316  0.000165      21      1.800811

In [179]:
zenodo_spreadsheet_files_df.groupby(['filetype']).agg({'filesize_mb': ['mean', 'min', 'max', 'median', 'count','sum']})

filesize_mb                                                         
                mean       min           max     median   count           sum
filetype                                                                     
.csv       63.839064  0.000000  98335.301978   0.142883  173074  1.104888e+07
.ods       21.392543  0.000037  19981.392191   0.034651    1257  2.689043e+04
.ogw        0.244249  0.017788      1.516737   0.170595      25  6.106224e+00
.ots        1.407787  1.407787      1.407787   1.407787       1  1.407787e+00
.tsv       73.102256  0.000003  34331.331091   0.178152    8894  6.501715e+05
.xls       35.244560  0.000000  20285.175328   0.055664    7349  2.590123e+05
.xlsb      14.863020  0.006146    110.212997  15.652452      97  1.441713e+03
.xlsm     106.095733  0.000152  18549.230797   9.098792     981  1.040799e+05
.xlsx      44.284170  0.000000  56269.237864   0.050755   88930  3.938191e+06
.xltm       0.366812  0.139694      2.537253   0.174035      12  4.401750e+00
.xltx      87.810956  0.001680   1827.139832   0.169394      21  1.844030e+03

In [127]:
zenodo_spreadsheet_files_df.groupby(['filetype']).agg({'filesize_gb': ['sum']})

,filesize_gb
,sum
filetype,
.csv,10789.924066
.ods,26.260183
.ogw,0.005963
.ots,0.001375
.tsv,634.933069
.xls,252.941670
.xlsb,1.407923
.xlsm,101.640541


In [180]:
zenodo_spreadsheet_files_df.groupby(['filetype']).agg({'filesize_gb': ['count']})

,filesize_gb
,count
filetype,
.csv,173074
.ods,1257
.ogw,25
.ots,1
.tsv,8894
.xls,7349
.xlsb,97
.xlsm,981


In [128]:
#https://stackoverflow.com/questions/41286569/get-total-of-pandas-column
spreadsheet_filesize_total = zenodo_spreadsheet_files_df['filesize'].sum()
print(spreadsheet_filesize_total)

6196192546123


In [129]:
#https://www.knowprogram.com/python/bytes-to-kb-mb-gb-and-tb-in-python/
#https://blog.finxter.com/5-best-ways-to-format-bytes-to-gigabytes-in-python/
#show the total size of all files, displaying in bytes, TB, and GB

terabytes_size = spreadsheet_filesize_total / (1024 * 1024 * 1024 * 1024)
formatted_tb_size = "{:.2f} TB".format(terabytes_size)
gigabytes_size = spreadsheet_filesize_total / (1024 ** 3)
formatted_gb_size = f"{gigabytes_size:.2f} GB"
print(formatted_tb_size)
print(formatted_gb_size)

5.64 TB
5770.65 GB


In [130]:
zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filetype'] == '.XL']

,fileid,filekey,filesize,filechecksum,filelinks.self,id,filetype,filename_noext,filesize_mb,filesize_gb


In [131]:
zenodo_files_all_df_sorted = zenodo_files_all_df.sort_values(by=['id','filename_noext','filetype'])

In [132]:
#https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
zenodo_files_all_df_sorted.groupby(['id','filename_noext'], sort=False).agg({'filename_noext':'count', 'filetype': ', '.join})

filename_noext filetype
id       filename_noext                                          
6957     SpanishTwitterMedia-JUN2013                   1    .xlsx
6966     CanalUGR                                      1     .csv
7027     CompactResults                                1    .xlsx
7068     ASprofile.tar                                 1      .gz
         BodyMap                                       1    .xlsx
...                                                  ...      ...
14616677 data_casestudy                                1     .rar
         settings2D                                    1     .csv
15046283 Guatemala_LasLisasALL_English                 1    .xlsm
         Guatemala_MonterricoALL_English               1    .xlsm
         PICES_7_well-being                            1    .xlsx

[739517 rows x 2 columns]

In [133]:
filename_counts = zenodo_files_all_df_sorted.groupby(['id','filename_noext'], sort=False).agg({'filename_noext':'count', 'filetype': ', '.join})

In [134]:
filename_counts.columns

Index(['filename_noext', 'filetype'], dtype='object')

In [135]:
filename_counts.index.name = None
filename_counts.columns=['counts', 'filetype']

In [136]:
filename_counts = filename_counts.rename_axis(['id','filename_noext']).reset_index()

In [137]:
filename_counts

,id,filename_noext,counts,filetype
0,6957,SpanishTwitterMedia-JUN2013,1,.xlsx
1,6966,CanalUGR,1,.csv
2,7027,CompactResults,1,.xlsx
3,7068,ASprofile.tar,1,.gz
4,7068,BodyMap,1,.xlsx
...,...,...,...,...
739512,14616677,data_casestudy,1,.rar
739513,14616677,settings2D,1,.csv
739514,15046283,Guatemala_LasLisasALL_English,1,.xlsm
739515,15046283,Guatemala_MonterricoALL_English,1,.xlsm


In [138]:
filename_duplicates = filename_counts.loc[filename_counts['counts']>1]

In [139]:
filename_duplicates

,id,filename_noext,counts,filetype
24,7582,PRL,4,".csv, .hproj, .rda, .txt"
25,7582,PRM,4,".csv, .hproj, .rda, .txt"
54,9979,NAO,2,".csv, .xlsx"
90,10854,Artikel_Sample,2,".bib, .ris"
96,10854,Buecher_sample_IBSS,2,".bib, .ris"
...,...,...,...,...
739234,14583147,Q6780799,2,".csv, .json"
739346,14597510,EMME24cat,2,".pdf, .xlsx"
739422,14598801,Year 2013,2,".pdf, .txt"
739423,14598801,Year 2014,2,".pdf, .txt"


In [140]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

filename_duplicates.to_csv(f'filename_duplicates_{filetime}.csv', index=False)

In [144]:
filename_duplicates['filetype'].str.contains('csv')

24         True
25         True
54         True
90        False
96        False
          ...  
739234     True
739346    False
739422    False
739423    False
739424    False
Name: filetype, Length: 44201, dtype: bool

In [151]:
#https://note.nkmk.me/en/python-pandas-str-contains-match/
figure_filenames = zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext'].str.match('(fig|figure)(\s|\W|_|-)',case=False)]

In [152]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

figure_filenames.to_csv(f'figure_filenames_{filetime}.csv', index=False)

In [156]:
zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext'].str.match('documentation',case=False)]

,fileid,filekey,filesize,filechecksum,filelinks.self,id,filetype,filename_noext,filesize_mb,filesize_gb
345261,1dfad98f-3334-4f6f-8570-d69f1d5ffe9e,Documentation_initialisation_process.xlsx,199385,md5:900abfc86dce6bf21608eb591dd0f659,https://zenodo.org/api/records/6782179/files/D...,6782179,.xlsx,Documentation_initialisation_process,0.190148,0.000186
672286,f8073947-6989-4eb4-9a55-00b2e06f6132,Documentation for Spatial dataset.xlsx,78006,md5:9f47317f90eef40f2e7ee15cf9135ef3,https://zenodo.org/api/records/10651059/files/...,10651059,.xlsx,Documentation for Spatial dataset,3.173740,0.003099
672287,e3c0e65a-2733-423f-a5aa-9b224e269268,Documentation for scRNA-seq dataset.xlsx,17772,md5:a93b12b3829ac5aeb3acaf06839cc7a8,https://zenodo.org/api/records/10651059/files/...,10651059,.xlsx,Documentation for scRNA-seq dataset,8.390197,0.008194
693088,af5087a5-acc0-489c-a989-fb7a94e63195,documentationmetrics.xlsx,67180,md5:b7a13365669abb0b4fa3f47ca0c43290,https://zenodo.org/api/records/11398627/files/...,11398627,.xlsx,documentationmetrics,0.005506,0.000005
736146,41aba47e-03b3-4eb7-91e0-16d34c73dbd4,Documentation_DataSets.xlsx,100686,md5:fc21cd4d9e47af6e6cb601f610d45bd1,https://zenodo.org/api/records/14356379/files/...,14356379,.xlsx,Documentation_DataSets,59.556365,0.058161


In [161]:
zenodo_spreadsheet_files_df.loc[zenodo_spreadsheet_files_df['filename_noext'].str.contains('[ \.\$@%#&\*\(\)\!]',case=False)]

,fileid,filekey,filesize,filechecksum,filelinks.self,id,filetype,filename_noext,filesize_mb,filesize_gb
9,603f592e-1567-4bfa-bac6-5f5f89a930f4,SUNNY SAM BEST RESULTS ON ONE PAGE HAWAII 2018...,135532436,md5:f21860be371093e02a07771b10ad7caf,https://zenodo.org/api/records/2596819/files/S...,2596819,.xlsx,SUNNY SAM BEST RESULTS ON ONE PAGE HAWAII 2018...,129.253803,0.126224
10,2d7f6725-d21b-4466-930e-3f470d10efb4,SMOKE VS OZONE BREWER DATA bulletin-119-2018-0...,26624,md5:52876404f04ec3a2b36a0db9fa0efede,https://zenodo.org/api/records/2596819/files/S...,2596819,.xls,SMOKE VS OZONE BREWER DATA bulletin-119-2018-0...,0.025391,0.000025
11,a95df4ed-696c-4f30-b4e7-c76145e9817d,SMOKE VS OZONE BREWER DATA bulletin-119-2018-0...,37430,md5:86cd14645ea618377a0ce0261681d9c2,https://zenodo.org/api/records/2596819/files/S...,2596819,.ods,SMOKE VS OZONE BREWER DATA bulletin-119-2018-0...,0.035696,0.000035
14,dab37f19-69d4-4522-bb54-11211eb8e875,MTOPS-BREWER CORRECTIONS.xlsx,18660,md5:d1793754653a128fd25720a02a72ae66,https://zenodo.org/api/records/2596819/files/M...,2596819,.xlsx,MTOPS-BREWER CORRECTIONS,0.017796,0.000017
16,8d9344e2-77be-4fca-a906-bea4b7e24341,CSU_BREWER Coimparison for 08072018.xlsx,33153,md5:2f5113c5671cefb1f7ec47202a71c7f6,https://zenodo.org/api/records/2596819/files/C...,2596819,.xlsx,CSU_BREWER Coimparison for 08072018,0.031617,0.000031
...,...,...,...,...,...,...,...,...,...,...
794634,1fba9d0f-d7db-4d0a-a14e-637cac649a1c,Input data NZ_CO_scenario.xlsm,43534090,md5:62a7e9eb8bb84a030efd15b9fc54aaa8,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_CO_scenario,1.516737,0.001481
794638,eb5ef863-d3ba-47f6-ae32-29178d0fedbc,Input data NZ_OL_scenario.xlsm,43532603,md5:8d4640c4e355150141c1d2ff9b43b3c8,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_OL_scenario,1.152891,0.001126
794639,63fe0822-4b74-4561-a625-6cb94d18160d,Input data NZ_scenario.xlsm,43532584,md5:6e10cf247a8e489af11e8d2b1955bc68,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_scenario,1.152891,0.001126
794644,46d84edb-7af1-4929-ba01-5ab6c53b2c26,Input data NZ_CC_scenario.xlsm,43534008,md5:233150a881b38d7a8684acc42dbc8765,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_CC_scenario,0.075867,0.000074


In [163]:
excel_ods_filetypes = ['.xls','.xlsb','.xlsx','.xltx','.ods','.ots','.xlsm','.xltm','.xltx']

In [165]:
#https://stackoverflow.com/questions/19960077/how-to-filter-pandas-dataframe-using-in-and-not-in-like-in-sql
excel_ods_files = zenodo_spreadsheet_files_df.query('filetype in @excel_ods_filetypes')

In [166]:
excel_ods_files

,fileid,filekey,filesize,filechecksum,filelinks.self,id,filetype,filename_noext,filesize_mb,filesize_gb
0,e28f3dc8-28bb-4c5e-9279-2ec8323baecd,database_D.xls,41472,md5:2bcfd1bd8dd9c49e8529bf6b1e2c0da8,https://zenodo.org/api/records/883073/files/da...,883073,.xls,database_D,0.039551,0.000039
2,a2414a60-1598-4781-9598-c6e490c45abb,database_D.ods,24710,md5:acd49bf242066b404136f75358735f1f,https://zenodo.org/api/records/883073/files/da...,883073,.ods,database_D,0.023565,0.000023
3,c3abd72d-8ff6-47d5-b54a-2a2c8ec8a37c,migration_modelling.xlsx,1193750,md5:1ffd4ff4009cb149e92b1faed928ca32,https://zenodo.org/api/records/883073/files/mi...,883073,.xlsx,migration_modelling,1.138449,0.001112
4,dd6092e9-477f-4309-a2dd-8a3d6fc4a950,migration_modelling.ods,958046,md5:73dcaae0470108cb4d26bf279a007a8a,https://zenodo.org/api/records/883073/files/mi...,883073,.ods,migration_modelling,0.913664,0.000892
6,1fe5b1ee-f3bb-45ac-ac3e-a8ad82169e67,05_Supplementary_table_1.xls,896512,md5:0617b7911ab69f9426e7f58e9817b235,https://zenodo.org/api/records/12918528/files/...,12918528,.xls,05_Supplementary_table_1,0.854980,0.000835
...,...,...,...,...,...,...,...,...,...,...
794634,1fba9d0f-d7db-4d0a-a14e-637cac649a1c,Input data NZ_CO_scenario.xlsm,43534090,md5:62a7e9eb8bb84a030efd15b9fc54aaa8,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_CO_scenario,1.516737,0.001481
794638,eb5ef863-d3ba-47f6-ae32-29178d0fedbc,Input data NZ_OL_scenario.xlsm,43532603,md5:8d4640c4e355150141c1d2ff9b43b3c8,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_OL_scenario,1.152891,0.001126
794639,63fe0822-4b74-4561-a625-6cb94d18160d,Input data NZ_scenario.xlsm,43532584,md5:6e10cf247a8e489af11e8d2b1955bc68,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_scenario,1.152891,0.001126
794644,46d84edb-7af1-4929-ba01-5ab6c53b2c26,Input data NZ_CC_scenario.xlsm,43534008,md5:233150a881b38d7a8684acc42dbc8765,https://zenodo.org/api/records/7859482/files/I...,7859482,.xlsm,Input data NZ_CC_scenario,0.075867,0.000074


In [177]:
#https://stackoverflow.com/questions/41286569/get-total-of-pandas-column
excel_ods_filesize_total = excel_ods_files['filesize'].sum()
print(excel_ods_filesize_total)

282241060676


In [178]:
#https://www.knowprogram.com/python/bytes-to-kb-mb-gb-and-tb-in-python/
#https://blog.finxter.com/5-best-ways-to-format-bytes-to-gigabytes-in-python/
#show the total size of all files, displaying in bytes, TB, and GB

terabytes_size = excel_ods_filesize_total / (1024 * 1024 * 1024 * 1024)
formatted_tb_size = "{:.2f} TB".format(terabytes_size)
gigabytes_size = excel_ods_filesize_total / (1024 ** 3)
formatted_gb_size = f"{gigabytes_size:.2f} GB"
print(formatted_tb_size)
print(formatted_gb_size)

0.26 TB
262.86 GB


In [167]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

excel_ods_files['id'].to_csv(f'excel_ods_{filetime}.csv', index=False)

In [175]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

zenodo_spreadsheet_files_df['id'].to_csv(f'zenodo_spreadsheet_files_{filetime}.csv', index=False)

In [ ]:
#to do length of filename

In [168]:
zenodo_spreadsheet_files_df_sample = zenodo_spreadsheet_files_df.sample(n=100, random_state=1)

In [169]:
zenodo_spreadsheet_files_df_sample

,fileid,filekey,filesize,filechecksum,filelinks.self,id,filetype,filename_noext,filesize_mb,filesize_gb
596073,e433705a-5396-493e-89e4-441f089f0c6b,Peel_Dublin_Jun85_009_Jun_17-05_00.csv,22653,md5:0deff73c0772e0d9a84c92f5f28d676b,https://zenodo.org/api/records/8305314/files/P...,8305314,.csv,Peel_Dublin_Jun85_009_Jun_17-05_00,0.028873,0.000028
571463,f8a15d46-bae4-494f-911e-26536a9eb639,Dublin_Llanbedrgoch_Jun05_024_Jun_02-05_00.csv,9740,md5:0159468a50edd845a0315895f8f84ac1,https://zenodo.org/api/records/8306624/files/D...,8306624,.csv,Dublin_Llanbedrgoch_Jun05_024_Jun_02-05_00,0.009289,0.000009
655258,5c26bc0d-9522-44e9-9673-6e5a5cf40523,Courses_CINE_list.csv,2528661,md5:cb0252417065758a261f57fdcc45d581,https://zenodo.org/api/records/11216882/files/...,11216882,.csv,Courses_CINE_list,1.710736,0.001671
482427,3c3968ba-32c7-4b89-9eed-449e753cff01,brca_RNA.csv,191750969,md5:d585358e54103aa3b182f5ea0632a84e,https://zenodo.org/api/records/8054482/files/b...,8054482,.csv,brca_RNA,182.867974,0.178582
93786,def3329f-8cc2-4977-ab6d-dba09af4c7ec,Figure 5.xls,28672,md5:7ff2067bf294d0aa57218b06d041da6d,https://zenodo.org/api/records/5464149/files/F...,5464149,.xls,Figure 5,0.027344,0.000027
...,...,...,...,...,...,...,...,...,...,...
12716,c658fb8b-af94-47cf-896f-5f18f589c948,Chloride Grabs and QA.xlsx,51169,md5:3e9c87b6b87973a1a6221843509e0a9e,https://zenodo.org/api/records/3458258/files/C...,3458258,.xlsx,Chloride Grabs and QA,0.048799,0.000048
749293,7916b26a-97b7-46ed-acc2-f9318726c6b9,SouceData_NatComm_ESI_NCOMMS-24-32410A.xlsx,39145971,md5:c06099ff56ee8bdf567beae8b799e542,https://zenodo.org/api/records/14022139/files/...,14022139,.xlsx,SouceData_NatComm_ESI_NCOMMS-24-32410A,0.216298,0.000211
382448,3fb0b5bc-2954-4d5b-9bfe-a9632631d83e,2018_06_13_CaWO4_002_8K_angle.csv,884340,md5:ea9de375af9827a8a1ce187d44483d3f,https://zenodo.org/api/records/5925657/files/2...,5925657,.csv,2018_06_13_CaWO4_002_8K_angle,0.843372,0.000824
588296,83d6d9aa-24ac-48a3-8ec1-875593e8d8e8,Peel_Dublin_Dec21_010_Dec_16-05_00.xlsx,29623,md5:09611189fe9106da5088396e136dc0b3,https://zenodo.org/api/records/8305766/files/P...,8305766,.xlsx,Peel_Dublin_Dec21_010_Dec_16-05_00,2.173334,0.002122


In [170]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

zenodo_spreadsheet_files_df_sample['id'].to_csv(f'zenodo_spreadsheet_files_df_sample_{filetime}.csv', index=False)

In [172]:
excel_ods_files_sample = excel_ods_files.sample(n=100, random_state=1)

In [173]:
excel_ods_files_sample

,fileid,filekey,filesize,filechecksum,filelinks.self,id,filetype,filename_noext,filesize_mb,filesize_gb
619140,61afb652-6d1a-4ab6-81c5-5826481c0b73,ExtendedDataFigure04_b.xlsx,58549,md5:826eeacac063095a98165a6bba17118b,https://zenodo.org/api/records/8121595/files/E...,8121595,.xlsx,ExtendedDataFigure04_b,2.637739,0.002576
113332,8bd2d1a1-a38e-4691-a5db-c814d90fd03e,FCCdb_201130_v5_Zenodo.xlsx,7972686,md5:aab1c6bcf24ee1b091b805af7ca09989,https://zenodo.org/api/records/4296944/files/F...,4296944,.xlsx,FCCdb_201130_v5_Zenodo,7.603346,0.007425
744586,f828b4b9-1332-40ec-b7bb-6862e864174c,Python_evaluation.xlsx,383900,md5:49d3f987e576c386a17214bb8b9400eb,https://zenodo.org/api/records/13837722/files/...,13837722,.xlsx,Python_evaluation,21.553707,0.021049
507381,63b62d22-bc99-4ac7-96a9-d5ea21e6f536,DEE_annual_data_nswph_nuts3_level_EL.xlsx,291289,md5:2651b5cf8b15d6e5cb48273eae3b5bf9,https://zenodo.org/api/records/7963195/files/D...,7963195,.xlsx,DEE_annual_data_nswph_nuts3_level_EL,0.277795,0.000271
588557,39e157cd-0692-4401-a8f1-2f6e25cd9c79,Dublin_Iona_Dec21_021_Dec_05-05_00.xlsx,35980,md5:b9a692b3a16c40a1b2f5313df71e3f1c,https://zenodo.org/api/records/8305766/files/D...,8305766,.xlsx,Dublin_Iona_Dec21_021_Dec_05-05_00,0.031299,0.000031
...,...,...,...,...,...,...,...,...,...,...
508586,97dc58a2-caee-4a69-ad8b-b461b0c8f1c2,CV 2M LiTFSI-DOL.xlsx,84831,md5:dff24f43856fff4142b085453013dc46,https://zenodo.org/api/records/8050623/files/C...,8050623,.xlsx,CV 2M LiTFSI-DOL,0.080901,0.000079
575908,fa0da2dd-67d0-47c3-a2af-bc7c7b68dc22,Iona_Dublin_Dec15_015_Dec_11-05_00.xlsx,31403,md5:d15c04ac1d3d3339db5452f013e48ffc,https://zenodo.org/api/records/8303177/files/I...,8303177,.xlsx,Iona_Dublin_Dec15_015_Dec_11-05_00,0.029948,0.000029
574757,0122d064-2720-416a-900c-ee52b339ee59,Chester_Dublin_Jun21_014_Jun_12-05_00.xlsx,29625,md5:e86a1733b857ed1c64d96c5497a25a6d,https://zenodo.org/api/records/8306908/files/C...,8306908,.xlsx,Chester_Dublin_Jun21_014_Jun_12-05_00,0.028253,0.000028
108822,82840441-117c-4e2d-acbb-11912eee985f,SupplementaryTable3.xlsx,1325992,md5:9cf1c85bbfb5f07c18c62c04caed3277,https://zenodo.org/api/records/4268051/files/S...,4268051,.xlsx,SupplementaryTable3,1.264565,0.001235


In [174]:
filetime = datetime.now()
filetime = filetime.strftime('%Y-%m-%d_%I-%M_%p')

excel_ods_files_sample['id'].to_csv(f'excel_ods_files_sample_{filetime}.csv', index=False)